In [1]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import itertools

In [2]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

In [3]:
houses = "https://www.domain.com.au/sale/eastern-suburbs-vic/?sort=price-asc&amp;page=1"
response = get(houses, headers=headers)

In [4]:
html_soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
house_containers = html_soup.find_all('div', class_="css-1gkcyyc")

In [6]:
prices = []
addresses = []
suburbs = []
states = []
postcodes = []
bedss = []
bathrooms = []
parkings = []
types = []
urls = []

In [7]:
%%time
n_pages = 0
for page in range(1,50):
    n_pages += 1
    houses_url = 'https://www.domain.com.au/sale/eastern-suburbs-vic/?ptype=duplex,free-standing,new-home-designs,new-house-land,semi-detached,terrace,town-house,villa&price=250000-750000&sort=price-asc'+ '&page='+str(page)
    r = get(houses_url, headers=headers)
    page_html = BeautifulSoup(r.text, 'html.parser')
    house_containers = page_html.find_all('div', class_="css-1gkcyyc")
    if house_containers != []:
        for container in house_containers:            
             #Price
            price = container.find_all('p')[0].text[0:9]
            price = price.replace(',','')
            price = price.replace('$','')
            price = price.replace(' ','')
            prices.append(price)
            #Address
            address = container.find_all('span')[0].text
            address = address.replace(',\xa0','')
            addresses.append(address)
            #Suburb
            suburb = container.find_all('span')[2].text
            suburbs.append(suburb)
            #State
            state = container.find_all('span')[3].text
            states.append(state)
            #Postcode
            postcode = container.find_all('span')[4].text
            postcodes.append(postcode)
            #Beds
            beds = container.find_all('span')[5].text
            beds = beds.replace(' Beds', '')
            beds = beds.replace(' Bed', '')
            beds = beds.replace('âˆ’', '')
            bedss.append(beds)
            
            #Bathrooms
            bathroom = container.find_all('span')[8].text
            bathroom = bathroom.replace(' Bath', '')
            bathroom = bathroom.replace('s', '')
            bathroom = bathroom.replace('âˆ’', '')
            bathrooms.append(bathroom)
            
            #Parking Spots
            parking = container.find_all('span')[11].text
            parking = parking.replace(' Parking', '')
            parking = parking.replace('âˆ’', '')
            parkings.append(parking)
            #Url
            url = container.find_all('a')[0].get('href')
            urls.append(url)
            #Type
#            type = container.find_all('span')[14].text
 #           types.append(type)
            
    else:
        break
        
    #sleep(randint(1,2))
    
print('You scraped {} pages containing {} properties.'.format(n_pages, len(addresses)))
print(houses_url)

You scraped 29 pages containing 482 properties.
https://www.domain.com.au/sale/eastern-suburbs-vic/?ptype=duplex,free-standing,new-home-designs,new-house-land,semi-detached,terrace,town-house,villa&price=250000-750000&sort=price-asc&page=29
Wall time: 16.3 s


In [8]:
cols = ['Price', 'Address', 'Suburb', 'State', 'Postcode', 'Beds', 'Bathrooms', 'Parking Spots', 'URL']
df = pd.DataFrame({'Price':prices, 
                   'Address': addresses, 
                   'Suburb': suburbs, 
                   'State': states, 
                   'Postcode': postcodes, 
                   'Beds': bedss, 
                   'Bathrooms': bathrooms, 
                   'Parking Spots': parkings, 'URL': urls})
df = df.drop_duplicates(subset='Address', keep='first')
df.head(100)

,Price,Address,Suburb,State,Postcode,Beds,Bathrooms,Parking Spots,URL
0,299000,105/31-43 Victoria Street,DONCASTER,VIC,3108,1,1,1,https://www.domain.com.au/105-31-43-victoria-s...
1,308750,20/74 Warrandyte,RINGWOOD,VIC,3134,1,1,1,https://www.domain.com.au/20-74-warrandyte-rin...
2,684500,4/21 English Street,SEVILLE,VIC,3139,4,2,2,https://www.domain.com.au/4-21-english-street-...
3,350000,Shop 19B/50 Dorset Square,BORONIA,VIC,3155,1,1,700,https://www.domain.com.au/shop-19b-50-dorset-s...
4,365000,211/31-43 Victoria Street,DONCASTER,VIC,3108,1,1,1,https://www.domain.com.au/211-31-43-victoria-s...
...,...,...,...,...,...,...,...,...,...
95,550000,50 Lomond Avenue,KILSYTH,VIC,3137,3,1,1,https://www.domain.com.au/50-lomond-avenue-kil...
96,549000,3/28 Mountain View Road,KILSYTH,VIC,3137,2,1,2,https://www.domain.com.au/3-28-mountain-view-r...
97,FORSALE,2/38 Wilsons Lane,LILYDALE,VIC,3140,2,2,1,https://www.domain.com.au/2-38-wilsons-lane-li...
98,550000,4/49 Albert Hill Road,LILYDALE,VIC,3140,3,2,2,https://www.domain.com.au/4-49-albert-hill-roa...


In [9]:
compression_opts = dict(method='zip',
                        archive_name='available_houses.csv')  
df.to_csv('available_houses.zip', index=False,
          compression=compression_opts)  